In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from magenta.models.music_vae import configs

import numpy as np
from magenta.models.music_vae.trained_model import TrainedModel # We need this class

def build_standalone_encoder(hparams, input_shape):
    """
    Builds a standard Keras model that replicates the custom BidirectionalLstmEncoder.
    
    Args:
        hparams: The same hparams object used by the original model.
        input_shape: The shape of a single input sequence (e.g., (96, 64)).
    Returns:
        A tf.keras.Model instance that replicates MusicVAE's encoder architecture in TF2 style
        and as a second return value, the graph containing the model.
    """

    encoder_graph = tf.Graph()
    with encoder_graph.as_default():
    
        # 1. Define the input layer
        input_tensor = tf.keras.layers.Input(shape=input_shape, name='encoder_input')
    
        # 2. Re-create the stacked LSTM cells as Keras layers
        # The original code uses a shared cell for fw and bw. The modern equivalent
        # is to create one stack of LSTM layers and wrap it in Bidirectional.
    
        # Start with the input tensor
        x = input_tensor
    
        # Create the stack of LSTM layers
        # The `return_sequences=True` is crucial for all but the last layer in a stack.
        num_lstm_layers = len(hparams.enc_rnn_size)
        for i, layer_size in enumerate(hparams.enc_rnn_size):
            lstm_layer = tf.keras.layers.LSTM(
                units=layer_size, 
                return_sequences=False, # The last (and only) LSTM returns a single vector
                name='multi_rnn_cell/cell_0/lstm_cell', # CRITICAL: Match TF1 variable scope
                recurrent_dropout=1.0 - hparams.dropout_keep_prob # To match the legacy DropoutWrapper
            )
        
            bi_lstm = tf.keras.layers.Bidirectional(
                lstm_layer,
                name='bidirectional_rnn' # CRITICAL: Match TF1 variable scope
            )

            # To create the 'cell_0' scope, we wrap the Bidirectional layer in a tiny sub-model
            # This is the key to matching the checkpoint's variable names.
            bi_lstm_model = tf.keras.Model(inputs=x, outputs=bi_lstm(x), name=f'cell_{i}')
            x = bi_lstm_model(x)

        # 3. Replicate the flatten operation
        # The original code flattens, but since the last LSTM returns a single vector per direction,
        # the output of Bidirectional is already "flat" in the time dimension.
    
        # 4. Re-create the final Dense layers
        # This layer needs to be inside an 'encoder' scope, which we will achieve by wrapping the whole thing.
        mu_layer = tf.keras.layers.Dense(hparams.z_size, name='mu')
    
        # The final output of our model is just the embedding (mu)
        mu_output = mu_layer(x)
    
        # 5. Create and return the final Keras model
        # We name the final model 'encoder' to add the last required scope.
        encoder_model = tf.keras.Model(inputs=input_tensor, outputs=mu_output, name='encoder')
    
        print("--- Standalone Keras Encoder Built with FINAL Corrected Naming ---")
        encoder_model.summary()
    
    return encoder_model, encoder_graph


def standalone_encoder_with_weight_copy(music_vae_trained_model,music_vae_model_config):
    """
    Instantiates a standalone TF2-style encore that 
    contains the approximative function and weights of 
    the MusicVAE encoder, which is in TF1 format
    
    Args:
        music_vae_trained_model: a TrainedModel instance.
        music_vae_model_config: The configuration associated with the TrainedModel
    """
    sess = music_vae_trained_model._sess
    
    print("Step 1: Successfully obtained session from original model")

    print("\n--- Step 2: Building the new standalone TF2 Keras encoder ---")
    encoder_input_shape = (music_vae_model_config.hparams.max_seq_len, music_vae_model_config.data_converter.input_depth)
    standalone_encoder, standalone_encoder_graph = build_standalone_encoder(music_vae_model_config.hparams, input_shape=encoder_input_shape)

    with standalone_encoder_graph.as_default():

        print("\n--- Step 3: Starting weight transfer from TF1 session to TF2 model ---")
        tf1_variables = sess.graph.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        tf1_variable_names = [v.name for v in tf1_variables]
        tf1_variable_values = sess.run(tf1_variables)
        tf1_weights_map = dict(zip(tf1_variable_names, tf1_variable_values))

        # Iterate through the layers of our new Keras model to set their weights
        for layer in standalone_encoder.layers:
            if layer.name == 'cell_0': # This is our Bidirectional sub-model
                for sub_layer in layer.layers:
                    if isinstance(sub_layer, tf.keras.layers.Bidirectional):
                        print(f"\nProcessing layer: {layer.name}/{sub_layer.name}")
                
                        fw_kernel_name = 'encoder/cell_0/bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel:0'
                        fw_bias_name = 'encoder/cell_0/bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias:0'
                        bw_kernel_name = 'encoder/cell_0/bidirectional_rnn/bw/multi_rnn_cell/cell_0/lstm_cell/kernel:0'
                        bw_bias_name = 'encoder/cell_0/bidirectional_rnn/bw/multi_rnn_cell/cell_0/lstm_cell/bias:0'
                
                        try:
                            # --- WEIGHT RE-ORDERING LOGIC ---
                            def reorder_lstm_weights(kernel, bias, num_units):
                                # TF1 format: [i, c, f, o] (input, cell, forget, output)
                                # TF2 format: [i, f, c, o] (input, forget, cell, output)
                        
                                # Split into the 4 gate weights
                                k_i, k_c, k_f, k_o = np.split(kernel, 4, axis=-1)
                                b_i, b_c, b_f, b_o = np.split(bias, 4, axis=-1)

                                # The legacy LSTMCell adds a `forget_bias` of 1.0 by default.
                                # We must manually add it to the forget gate's bias.
                                b_f = b_f + 1.0
                        
                                # Re-assemble in TF2 order (swap c and f)
                                reordered_kernel = np.concatenate([k_i, k_f, k_c, k_o], axis=-1)
                                reordered_bias = np.concatenate([b_i, b_f, b_c, b_o], axis=-1)
                        
                                return reordered_kernel, reordered_bias

                            # Get weights from checkpoint
                            fw_kernel_tf1, fw_bias_tf1 = tf1_weights_map[fw_kernel_name], tf1_weights_map[fw_bias_name]
                            bw_kernel_tf1, bw_bias_tf1 = tf1_weights_map[bw_kernel_name], tf1_weights_map[bw_bias_name]

                            # Re-order weights to match Keras's expected format
                            num_units = music_vae_model_config.hparams.enc_rnn_size[0]
                            fw_kernel_tf2, fw_bias_tf2 = reorder_lstm_weights(fw_kernel_tf1, fw_bias_tf1, num_units)
                            bw_kernel_tf2, bw_bias_tf2 = reorder_lstm_weights(bw_kernel_tf1, bw_bias_tf1, num_units)

                            # Split the re-ordered kernels for Keras
                            input_depth = music_vae_model_config.data_converter.input_depth
                            fw_input_kernel, fw_recurrent_kernel = np.split(fw_kernel_tf2, [input_depth], axis=0)
                            bw_input_kernel, bw_recurrent_kernel = np.split(bw_kernel_tf2, [input_depth], axis=0)
                    
                            # Assemble the final list of 6 weights
                            keras_weights = [
                                fw_input_kernel, fw_recurrent_kernel, fw_bias_tf2,
                                bw_input_kernel, bw_recurrent_kernel, bw_bias_tf2
                            ]
                    
                            sub_layer.set_weights(keras_weights)
                            print(f"  - Successfully re-ordered, split, and transferred 6 weights for Bidirectional LSTM.")

                        except Exception as e:
                            print(f"  - FATAL ERROR: An exception occurred during weight transfer: {e}")

            if layer.name == 'mu' and isinstance(layer, tf.keras.layers.Dense):
                # This part remains the same
                print(f"\nProcessing layer: {layer.name}")
                kernel_name = f'encoder/mu/kernel:0'
                bias_name = f'encoder/mu/bias:0'
                try:
                    layer.set_weights([tf1_weights_map[kernel_name], tf1_weights_map[bias_name]])
                    print(f"  - Successfully transferred weights for Dense layer 'mu'.")
                except KeyError as e:
                    print(f"  - FATAL ERROR: Could not find weight {e} for 'mu' layer!")

    return standalone_encoder, standalone_encoder_graph

def load_standalone_encoder(SAVED_ENCODER_PATH):
    """
    Loads a standalone Keras encoder model from disk into its own graph and session.
    
    Args:
        SAVED_ENCODER_PATH: Path to the saved Keras model (HDF5 or SavedModel format).
    """    


    # ======================================================================
    #  Load the Standalone Encoder into its own Graph and Session
    # ======================================================================

    # 1. Create a new, empty graph to serve as the blueprint for our loaded model.
    encoder_graph = tf.Graph()

    # 2. Create a new session that will be dedicated to this new graph.
    # We create it now so we can use it to initialize variables after loading.
    encoder_sess = tf.compat.v1.Session(graph=encoder_graph)

    # 3. Use the graph's context to load the model. This tells Keras where to
    #    rebuild the model's architecture.
    with encoder_graph.as_default():
        # Set the session context for Keras to use during loading.
        with encoder_sess.as_default():
            print(f"Loading standalone encoder from: {SAVED_ENCODER_PATH}")
        
            # Load the model. Keras will rebuild the layers on `encoder_graph`.
            loaded_standalone_encoder = tf.keras.models.load_model(SAVED_ENCODER_PATH)
        
            # IMPORTANT: Initialize all variables in the new session.
            # This finalizes the model loading process for the TF1 session.
            #encoder_sess.run(tf.compat.v1.global_variables_initializer())

    return loaded_standalone_encoder, encoder_graph, encoder_sess








In [ ]:
# Copy over the weights and save as TF2-style model

print("--- Step 1: Loading original TF1-style MusicVAE model ---")
mel_2bar_config = configs.CONFIG_MAP['cat-mel_2bar_big']
BASE_DIR = "models/download.magenta.tensorflow.org/models/music_vae"
checkpoint_path = BASE_DIR + '/checkpoints/mel_2bar_big.ckpt'
mel_2bar = TrainedModel(mel_2bar_config, batch_size=4, checkpoint_dir_or_path=checkpoint_path)
print("Original model loaded successfully.")

standalone_encoder, standalone_encoder_graph=standalone_encoder_with_weight_copy(mel_2bar,mel_2bar_config)



print("\n--- Weight transfer complete. Saving the new Keras model. ---")
output_keras_model_path = 'models/music_vae_encoder_keras'
tf.keras.models.save_model(standalone_encoder, output_keras_model_path)
print(f"\nSuccessfully saved the new Keras model to: '{output_keras_model_path}'")



In [ ]:
# Use tensorflow.compat.v1 and disable V2 behavior for the original model

# ==============================================================================
# 1. SETUP & MODEL LOADING
# ==============================================================================

print("--- Step 1: Loading original TF1-style MusicVAE model ---")
mel_2bar_config = configs.CONFIG_MAP['cat-mel_2bar_big']
BASE_DIR = "models/download.magenta.tensorflow.org/models/music_vae"
checkpoint_path = BASE_DIR + '/checkpoints/mel_2bar_big.ckpt'

# Use a batch size of 1 for easier comparison
BATCH_SIZE = 1
mel_2bar = TrainedModel(mel_2bar_config, batch_size=BATCH_SIZE, checkpoint_dir_or_path=checkpoint_path)
sess = mel_2bar._sess
print("Original model loaded.")

print("\n--- Step 2: Loading the new standalone Keras encoder ---")
keras_model_path = 'models/music_vae_encoder_keras'
# We can load the Keras model using the modern API
standalone_encoder, encoder_graph, encoder_sess =load_standalone_encoder(keras_model_path)
#standalone_encoder = tf.keras.models.load_model(keras_model_path)

print("New Keras model loaded.")


# ==============================================================================
# 2. INFERENCE & COMPARISON
# ==============================================================================

print("\n--- Step 3: Generating a random input tensor ---")
seq_len = mel_2bar_config.hparams.max_seq_len
input_depth = mel_2bar_config.data_converter.input_depth
control_depth = mel_2bar_config.data_converter.control_depth # This will be 0
input_shape = (BATCH_SIZE, seq_len, input_depth)

random_input = np.random.rand(*input_shape).astype(np.float32)
print(f"Generated random input with shape: {random_input.shape}")

print("\n--- Step 4: Running inference on both models ---")

# --- Get embedding from the ORIGINAL TF1 model ---

# ** THE FIX IS HERE **
# Create an empty array for the `_controls` placeholder
empty_controls = np.zeros((BATCH_SIZE, seq_len, control_depth), dtype=np.float32)
print(f"Generated empty controls with shape: {empty_controls.shape}")

# Add the empty controls to the feed_dict
feed_dict = {
    mel_2bar._inputs: random_input,
    mel_2bar._inputs_length: [seq_len] * BATCH_SIZE,
    mel_2bar._controls: empty_controls # Add the required empty placeholder value
}
# `_mu` is the tensor that holds the embedding
original_embedding = sess.run(mel_2bar._mu, feed_dict)


# --- Get embedding from the NEW Keras model ---

with encoder_graph.as_default():
    with encoder_sess.as_default():
        keras_embedding = standalone_encoder.predict(random_input)

#keras_embedding = standalone_encoder.predict(random_input)
# ==============================================================================
# 3. VERIFICATION
# ==============================================================================

print("\n--- Step 5: Comparing the outputs ---")
print(f"Original model's embedding (first 5 values): {original_embedding[0, :5]}")
print(f"New Keras model's embedding (first 5 values):  {keras_embedding[0, :5]}")

are_close = np.allclose(original_embedding, keras_embedding, atol=1e-6)

print("\n--- VERIFICATION RESULT ---")
if are_close:
    print("✅ SUCCESS: The embeddings from both models are identical!")
    abs_diff = np.mean(np.abs(original_embedding - keras_embedding))
    print(f"   (Mean absolute difference: {abs_diff:.10f})")
else:
    print("❌ FAILURE: The embeddings do not match.")
    abs_diff = np.mean(np.abs(original_embedding - keras_embedding))
    print(f"   (Mean absolute difference: {abs_diff})")



In [ ]:
import tensorflow as tf
import numpy as np

# Ensure we are in a TF1-compatible environment
tf.compat.v1.disable_eager_execution()

# --- Configuration ---
SAVED_ENCODER_PATH = 'models/music_vae_encoder_keras'
# Assume `mel_2bar` is your already-loaded original model object

# ======================================================================
#  1. SETUP: Prepare both models in their isolated sessions
# ======================================================================

# --- Model A: Original TF1 Model ---
sess_a = mel_2bar._sess
graph_a = sess_a.graph
print(f"Model A (Original) is ready in Session: {sess_a}")

# --- Model B: Loaded Keras Standalone Encoder ---
# Should already be loaded from above
#standalone_encoder, encoder_graph, encoder_sess = load_standalone_encoder(SAVED_ENCODER_PATH)
#print(f"Model B (Keras) is ready in Session: {encoder_sess}")


# ======================================================================
#  2. DEFINE TARGETS: Identify the tensors to compare
# ======================================================================

# --- Tensor names from Model A (TF1) ---
# (These are the names we found in previous steps)
TF1_INPUT_SEQ = "Placeholder_2:0"
TF1_INPUT_LEN = "Placeholder_1:0"
TF1_LSTM_OUTPUT = "encoder/cell_0/concat:0"
TF1_FINAL_STATE = "concat_2:0"
TF1_MU_OUTPUT = "encoder/mu/BiasAdd:0"

# --- Corresponding layers from Model B (Keras) ---
# (Use the names you defined in your Keras model. Use `standalone_encoder.summary()` to find them)
KERAS_LSTM_LAYER_NAME = 'cell_0' # The Bidirectional wrapper layer
KERAS_CONCAT_LAYER_NAME = 'final_state_concat' # The layer that concatenates the final states
KERAS_MU_LAYER_NAME = 'mu' # The final dense layer for mu

# ======================================================================
#  3. EXECUTE AND COMPARE: Run the layer-by-layer check
# ======================================================================



# --- Check 1: Bidirectional LSTM Sequence Output ---
print("\n--- Comparing Layer 1: Bi-LSTM Sequence Output ---")

# Fetch from Model A
with graph_a.as_default():
    tf1_lstm_out_val = sess_a.run(
        graph_a.get_tensor_by_name(TF1_LSTM_OUTPUT),
        feed_dict=feed_dict
    )

# Fetch from Model B
with encoder_graph.as_default():
    # The Bi-LSTM layer returns multiple tensors; the sequence output is the first one ([0])
    keras_lstm_output_tensor = standalone_encoder.get_layer(KERAS_LSTM_LAYER_NAME).output[0]
    keras_lstm_out_val = encoder_sess.run(
        keras_lstm_output_tensor,
        feed_dict={
            input_tensor: random_input
        }
    )

# Compare
try:
    np.testing.assert_allclose(tf1_lstm_out_val, keras_lstm_out_val, rtol=1e-5, atol=1e-5)
    print("✅ SUCCESS: Bi-LSTM outputs match.")
except AssertionError:
    print("❌ FAILURE: Bi-LSTM outputs DO NOT match. The problem is in your Bidirectional/LSTM layer configuration.")
    # If this fails, there's no need to check further layers.
    # The issue could be: different number of units, activation functions, dropout settings, etc.


# --- Check 2: Final Concatenated State ---
# (Only run this if the previous check passed)
print("\n--- Comparing Layer 2: Final Concatenated State ---")

# Fetch from Model A
with graph_a.as_default():
    tf1_state_val = sess_a.run(
        graph_a.get_tensor_by_name(TF1_FINAL_STATE),
        feed_dict={
            TF1_INPUT_SEQ: input_sequence,
            TF1_INPUT_LEN: sequence_length
        }
    )

# Fetch from Model B
with encoder_graph.as_default():
    keras_state_tensor = standalone_encoder.get_layer(KERAS_CONCAT_LAYER_NAME).output
    keras_state_val = encoder_sess.run(
        keras_state_tensor,
        feed_dict={
            standalone_encoder.input: input_sequence
        }
    )

# Compare
try:
    np.testing.assert_allclose(tf1_state_val, keras_state_val, rtol=1e-5, atol=1e-5)
    print("✅ SUCCESS: Final states match.")
except AssertionError:
    print("❌ FAILURE: Final states DO NOT match. The problem is how you extract and concatenate the LSTM states.")


# --- Check 3: Final `mu` Embedding ---
# (Only run this if the previous checks passed)
print("\n--- Comparing Layer 3: Final `mu` Embedding ---")

# Fetch from Model A
with graph_a.as_default():
    tf1_mu_val = sess_a.run(
        graph_a.get_tensor_by_name(TF1_MU_OUTPUT),
        feed_dict={
            TF1_INPUT_SEQ: input_sequence,
            TF1_INPUT_LEN: sequence_length
        }
    )

# Fetch from Model B
with encoder_graph.as_default():
    keras_mu_tensor = standalone_encoder.get_layer(KERAS_MU_LAYER_NAME).output
    keras_mu_val = encoder_sess.run(
        keras_mu_tensor,
        feed_dict={
            standalone_encoder.input: input_sequence
        }
    )

# Compare
try:
    np.testing.assert_allclose(tf1_mu_val, keras_mu_val, rtol=1e-5, atol=1e-5)
    print("✅ SUCCESS: The final `mu` embeddings match!")
except AssertionError:
    print("❌ FAILURE: The final `mu` embeddings DO NOT match. The problem is in the final Dense layer.")



